**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [1]:
# Datasets
import pandas as pd

# Modelado:
!pip install sentiment_analysis_spanish
!pip install transformers
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.5.0/es_core_news_lg-3.5.0.tar.gz
!pip install gliner
from gliner import GLiNER
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
import spacy
from sentiment_analysis_spanish import sentiment_analysis

# Configuración de warnings:
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Prompts

Ingreso del usuario mediante teclado de los prompts a evaluar por el programa.

In [2]:
def sentiment_input() -> str:
  """
  Inicia el programa y pide al usuario que ingrese un prompt.
  """
  print('¡Bienvenido al recomendador de actividades!')
  sentiment_input = input("¿Cómo te sientes hoy?: ")
  return sentiment_input

In [3]:
def preference_input() -> str:
  """
  Pide al usuario que ingrese su temática a explorar.
  """
  preference_input = input("¿Qué tipo de temática querés abordar?: ")
  return preference_input

## Modelado
- Se utiliza transformers para clasificar el estado de ánimo de la persona. Este puede ser: "Alegre", "Melancólico" o "Ni fu ni fa".

### Transformers

In [4]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        print(score)
        return 'Alegre'
    elif label == '3 stars':
        print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        print(score)
        return 'Melancólico'

In [6]:
test_list: list[str] = [
    "Estoy muy feliz",
    "Me siento entusiasmado",
    "Estoy tranquilo",
    "Me siento relajado",
    "Estoy aburrido",
    "Me siento nostálgico",
    "Estoy un poco triste",
    "Me siento melancólico",
    "Estoy frustrado",
    "Me siento ansioso",
    "Estoy enojado",
    "Me siento decepcionado",
    "Estoy muy emocionado",
    "Me siento inspirado",
    "Estoy cansado",
    "Me siento agotado",
    "Estoy preocupado",
    "Me siento optimista",
    "Estoy motivado",
    "Me siento agradecido",
    "Estoy desconcentrado",
    "Me siento esperanzado",
    "Estoy estresado",
    "Me siento insatisfecho",
    "Estoy satisfecho",
    "Me siento apático",
    "Estoy en paz",
    "Me siento irritado",
    "Estoy un poco nervioso",
    "Me siento confiado",
    "Estoy renovado",
    "Me siento pleno",
    "Estoy resignado",
    "Me siento inquieto",
    "Estoy contento",
    "Me siento vulnerable",
    "Estoy enamorado",
    "Me siento frustrado",
    "Estoy ilusionado",
    "Me siento reflexivo",
    "Estoy agradecido",
    "Me siento emocionalmente agotado"
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

0.7204190492630005
Estoy muy feliz -> Alegre
0.5104050636291504
Me siento entusiasmado -> Alegre
0.4654117822647095
Estoy tranquilo -> Alegre
0.3073946535587311
Me siento relajado -> Melancólico
0.6528457999229431
Estoy aburrido -> Melancólico
0.32323989272117615
Me siento nostálgico -> Alegre
0.5161958932876587
Estoy un poco triste -> Ni fu ni fa
0.3490581214427948
Me siento melancólico -> Melancólico
0.6465574502944946
Estoy frustrado -> Melancólico
0.45507076382637024
Me siento ansioso -> Alegre
0.6622191667556763
Estoy enojado -> Melancólico
0.469738245010376
Me siento decepcionado -> Melancólico
0.6982787847518921
Estoy muy emocionado -> Alegre
0.38369399309158325
Me siento inspirado -> Alegre
0.6665818095207214
Estoy cansado -> Melancólico
0.40258100628852844
Me siento agotado -> Melancólico
0.35649794340133667
Estoy preocupado -> Melancólico
0.30568787455558777
Me siento optimista -> Ni fu ni fa
0.4021136164665222
Estoy motivado -> Alegre
0.45900627970695496
Me siento agradecido

### sentiment_analysis_spanish library

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

El rendimiento obtenido es inferior, teniendo en cuenta que un resultado cercano a uno es un texto positivo, cercano a cero es negativo y en los valores medios es neutro. Por eso se optó por los transformers.

Para obtener esta conclusión se visitó la [Documentación](https://pypi.org/project/sentiment-analysis-spanish/).

In [7]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

Estoy muy feliz -> 0.6889487711193526
Me siento entusiasmado -> 0.0728752984896165
Estoy tranquilo -> 0.7826378949098491
Me siento relajado -> 0.23951579470116582
Estoy aburrido -> 0.31062133515766494
Me siento nostálgico -> 0.0728752984896165
Estoy un poco triste -> 0.04078651378014973
Me siento melancólico -> 0.0728752984896165
Estoy frustrado -> 0.31062133515766494
Me siento ansioso -> 0.0728752984896165
Estoy enojado -> 0.31062133515766494
Me siento decepcionado -> 0.005644903897282337
Estoy muy emocionado -> 0.537734459784737
Me siento inspirado -> 0.0728752984896165
Estoy cansado -> 0.31062133515766494
Me siento agotado -> 0.0728752984896165
Estoy preocupado -> 0.31062133515766494
Me siento optimista -> 0.0728752984896165
Estoy motivado -> 0.31062133515766494
Me siento agradecido -> 0.0728752984896165
Estoy desconcentrado -> 0.31062133515766494
Me siento esperanzado -> 0.0728752984896165
Estoy estresado -> 0.31062133515766494
Me siento insatisfecho -> 0.0728752984896165
Estoy sat

### NER

Se emplea para la identificación de entidades que facilitarán la búsqueda en los datos, entendiendo y analizando tanto sus preferencias como el contexto.

In [9]:
ner = spacy.load('es_core_news_lg')

In [12]:
for texto in test_list:
  doc = ner(texto)
  for ent in doc.ents:
    print(f'Entidad: {ent.text}, Etiqueta: {ent.label_}, Explicación: {spacy.explain(ent.label_)}')

Entidad: Estoy muy feliz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enojado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy cansado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy estresado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy en paz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enamorado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [21]:
!pip install gliner
from gliner import GLiNER

gliner_model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [24]:
gliner_model.eval()

labels: list[str] = ["person", "book", "location", "date", "actor", "character","game"]


for text in test_list:
  entities = gliner_model.predict_entities(text, labels, threshold=0.4)
  for entity in entities:
    print(entity, "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### POS

In [29]:
pos = spacy.load('es_core_news_lg')

data = []
# Procesar el texto con el modelo de spaCy
for text in test_list:
  doc = pos(texto)
  for token in doc:
      data.append([token.text, token.pos_, spacy.explain(token.pos_)])


df = pd.DataFrame(data, columns=['Palabra', 'Etiqueta POS', 'Explicación'])

print(df)

     Palabra Etiqueta POS               Explicación
0       Juan        PROPN               proper noun
1          y        CCONJ  coordinating conjunction
2      Pedro        PROPN               proper noun
3     fueron          AUX                 auxiliary
4         al          ADP                adposition
..       ...          ...                       ...
793    perro        PROPN               proper noun
794  buscaba         VERB                      verb
795       un          DET                determiner
796    hueso         NOUN                      noun
797        .        PUNCT               punctuation

[798 rows x 3 columns]
